# Importing Libraries

In [1]:
import datetime
import multiprocessing
import random
import time

import wandb
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CallbackList, EvalCallback
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage
from wandb.integration.sb3 import WandbCallback
from feature_extraction.callbacks.wandb_on_training_end_callback import WandbOnTrainingEndCallback
from feature_extraction.feature_extractors.resnet.block_feature_extractor import BlockFeatureExtractor
from feature_extraction.wrappers.vec_feature_extractor import VecFeatureExtractor
from utils import linear_schedule
from collections import OrderedDict

2024-03-28 00:43:44.922222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 00:43:44.922271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 00:43:44.923814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-28 00:43:44.933290: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 00:43:46.035027: W tensorflow/compiler/tf2

# Method for creating a new configuration

In [2]:
def create_config(project_name, run_name):
    """
    Creates a new configuration as an OrderedDict with a specified project and run name,
    and sets the training_seed and evaluation_seed to random integers,
    while using a predefined set of default settings for the rest.

    Parameters:
    - project_name (str): Name of the project.
    - run_name (str): Name of the run.

    Returns:
    - OrderedDict: A new configuration dictionary with the specified project and run names,
                   random seeds, and default settings for other parameters.
    """
    training_seed = random.randint(0, 9999) # Should later on be same for all experiments to ensure comparability
    evaluation_seed = random.randint(0, 9999) # Should later on be same for all experiments to ensure comparability

    default_config = OrderedDict([
        # Environment settings
        ('project_name', project_name),
        ('run_name', run_name),
        ('env_id', "ALE/Breakout-v5"),
        ('n_envs', 8),
        ('env_wrapper', ['stable_baselines3.common.atari_wrappers.AtariWrapper']),
        ('frame_stack', 4),
        ('training_seed', training_seed),
        ('evaluation_seed', evaluation_seed),
        
        # Algorithm and policy settings
        ('algo', 'PPO'),
        ('policy', 'CnnPolicy'),
        
        # Training hyperparameters
        ('batch_size', 256),
        ('n_steps', 128),
        ('n_epochs', 4),
        ('n_timesteps', 3_000),
        ('learning_rate', 0.00025),
        ('learning_rate_schedule', 'linear'),
        ('clip_range', 0.1),
        ('clip_range_schedule', 'linear'),
        ('ent_coef', 0.01),
        ('vf_coef', 0.5),
        ('normalize_advantage', False),
        
        # Evaluation and logging settings
        ('n_eval_episodes', 5),
        ('record_n_episodes', 10),
        ('log_frequency', 1024),
        
        # Other settings
        ('verbose', 1)
    ])
    

    return default_config

# Method for running a single experiment with a configuration

In [3]:
from torchvision.models import ResNet50_Weights
from torchvision import models


def run_experiment(config):
    wandb.login()



    # Initialize the wandb run
    wandb.init(project=config['project_name'],
                     name=config['run_name'],
                     config=config, save_code=True,
                     sync_tensorboard=True)
    
    

    config = wandb.config
    
    log_dir = f"logs/{config.run_name}"
    #feature_extractor = StageFeatureExtractor()
    model = models.resnet50(ResNet50_Weights.DEFAULT)
    feature_extractor = BlockFeatureExtractor(model, 1)

    # Create Evaluation Environment
    vec_eval_env = make_atari_env(config.env_id, n_envs=config.n_envs, seed=config.evaluation_seed)
    vec_eval_env = VecFrameStack(vec_eval_env, n_stack=config.frame_stack)
    vec_eval_env = VecTransposeImage(vec_eval_env)
    vec_eval_env = VecFeatureExtractor(vec_eval_env, feature_extractor)

    # Create Training Environment
    vec_train_env = make_atari_env(config.env_id, n_envs=config.n_envs, seed=config.training_seed)
    vec_train_env = VecFrameStack(vec_train_env, n_stack=config.frame_stack)
    vec_train_env = VecTransposeImage(vec_train_env)

    #

    # Define the keys for PPO-specific hyperparameters
    ppo_params_keys = [
        'batch_size',
        'ent_coef',
        'n_epochs',
        'n_steps',
        'policy',
        'vf_coef',
        'normalize_advantage',
    ]

    # Filter the config dictionary to extract only the PPO hyperparameters
    ppo_hyperparams = {key: config[key] for key in ppo_params_keys if key in config}

    # Additional hyperparameters not in the initial filter that require custom handling
    learning_rate_schedule = linear_schedule(2.5e-4)
    clip_range_schedule = linear_schedule(0.1)

    # Instantiate the PPO model with the specified hyperparameters and environment
    model = PPO(
        **ppo_hyperparams,
        learning_rate=learning_rate_schedule,
        clip_range=clip_range_schedule,
        env=vec_train_env,
        verbose=1,
        tensorboard_log=f"{log_dir}",
    )

    # Create Callbacks

    # Save best model
    eval_callback = EvalCallback(
        eval_env=vec_eval_env,
        eval_freq=max(config.log_frequency // config.n_envs, 1),
        n_eval_episodes=config.n_eval_episodes,
        best_model_save_path=log_dir,
        log_path=log_dir,
        deterministic=True,
        render=False,
        verbose=0
    )

    # Needs to be changed, so it uses run instead of wandb
    wandb_callback = WandbCallback(
        verbose=1,
        gradient_save_freq=256,
    )

    wandb_on_training_end_callback = WandbOnTrainingEndCallback(
        model=model,
        eval_env=vec_eval_env,
        log_dir=log_dir,
        n_eval_episodes=config.n_eval_episodes,
        record_n_episodes=config.record_n_episodes,
    )
    callbacks = CallbackList([wandb_callback, eval_callback, wandb_on_training_end_callback])

    model.learn(
        total_timesteps=config.n_timesteps,
        callback=callbacks,
    )

    wandb.finish()

# Method for running multiple experiments with a configuration

In [4]:
def run_experiments_in_parallel(config_list):
    processes = []
    for config in config_list:
        p = multiprocessing.Process(target=run_experiment, args=(config,))
        p.start()
        processes.append(p)
        time.sleep(60) # Sleep to avoid file write conflicts

    for p in processes:
        p.join()

# Run experiments

In [5]:
configs = []
runs = ["breakout_vec_feat"]

for run in runs:
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    
    project_name = "parallel_experiments"
    run_name = run + timestamp
    
    new_config = create_config(project_name, run_name)
    configs.append(new_config)

# Run experiments in parallel
run_experiments_in_parallel(configs)

wandb: Currently logged in as: adicreson (feature_extraction). Use `wandb login --relogin` to force relogin


/home/maliti/Prog/Global-Feature-Extraction/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/maliti/Prog/Global-Feature-Extraction/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Using cpu device
Logging to logs/breakout_vec_feat2024-03-28_00-43-48/PPO_1


/home/maliti/Prog/Global-Feature-Extraction/venv/lib/python3.10/site-packages/stable_baselines3/common/callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7fa13ad8d180> != <feature_extraction.wrappers.vec_feature_extractor.VecFeatureExtractor object at 0x7fa13ad8d690>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")
2024-03-28 00:43:56.856055: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 00:43:56.856103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 00:43:56.857188: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable

KeyboardInterrupt: 